<a href="https://colab.research.google.com/github/SWAROOPBHAGWAT/Crop-Prediction/blob/main/Crop%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import tree

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/dataset/Crop_recommendation.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/dataset/Crop_recommendation.csv'

### ***Data Analysis : EDA***

In [ ]:
df.head()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
df['label'].unique()

In [ ]:
df['label'].value_counts()

### **Graphs to Analyse the data**

In [ ]:

numeric_features = [feature for feature in df.columns if df[feature].dtype != 'O']

plt.figure(figsize=(20, 220))  # Increase the figure size

for i, col in enumerate(numeric_features):
    plt.subplot(60, 3, i+1)
    sns.distplot(x=df[col], color='indianred')
    plt.xlabel(col, weight='bold', fontsize=14)
    plt.ylabel('Density', weight='bold', fontsize=14)  # Increase the font size
    plt.xticks(fontsize=12)  # Increase the font size of tick labels on x-axis
    plt.yticks(fontsize=12)  # Increase the font size of tick labels on y-axis
    plt.title(f'Distribution of {col}', weight='bold', fontsize=16)  # Increase the font size
    plt.tight_layout()

plt.show()


In [ ]:
plt.figure(figsize=(25, 4))
sns.countplot(x=df["label"])
plt.show();

### **KDE Plot** || **Violin Plot** || **Box Plot**


In [ ]:
plt.style.use('ggplot')
sns.set_palette("hls", 8)
for i in df.columns[:-1]:
    fig,ax=plt.subplots(1,3,figsize=(18,4))
    sns.histplot(data=df,x=i,kde=True,bins=40,ax=ax[0])
    sns.violinplot(data=df,x=i,ax=ax[1])
    sns.boxplot(data=df,x=i,ax=ax[2])
    plt.suptitle(f'Visualizing {i}',size=20)

### **Heatmap**

In [ ]:
# Exclude non-numeric columns
numeric_columns = df.select_dtypes(include=['number'])

# Calculate correlation matrix
correlation_matrix = numeric_columns.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()


### **Pairplot**

In [ ]:
sns.pairplot(data=df,hue='label')
plt.show()



## **Seperating features and target label**

In [ ]:
features = df[['N', 'P','K','temperature', 'humidity', 'ph', 'rainfall']]
target = df['label']
labels = df['label']

In [ ]:
# Initializing empty lists to append all model's name and corresponding name
acc = []
model = []

## **Splitting the data into training and testing sets**

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size = 0.2,random_state =2)

## **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DecisionTree = DecisionTreeClassifier(criterion="entropy",random_state=2,max_depth=5)

DecisionTree.fit(Xtrain,Ytrain)

predicted_values = DecisionTree.predict(Xtest)
x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('Decision Tree')
print("DecisionTrees's Accuracy is: ", x*100)

print(classification_report(Ytest,predicted_values))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Ytest, predicted_values)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="OrRd", fmt="d", xticklabels=np.unique(Ytest), yticklabels=np.unique(Ytest))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.model_selection import cross_val_score

# Cross validation score (Decision Tree)
score = cross_val_score(DecisionTree, features, target,cv=5)
score


## **Guassian Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Assuming Xtrain, Ytrain, Xtest, Ytest are your training and test data

# Initialize and train Naive Bayes classifier
NaiveBayes = GaussianNB()
NaiveBayes.fit(Xtrain, Ytrain)

# Make predictions
predicted_values = NaiveBayes.predict(Xtest)

# Calculate accuracy and append to acc list
x = accuracy_score(Ytest, predicted_values)
acc.append(x)

# Append model name to the model list
model.append('Naive Bayes')

# Print accuracy and classification report
print("Naive Bayes's Accuracy is:", x)
print(classification_report(Ytest, predicted_values))



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


# Calculate confusion matrix
cm = confusion_matrix(Ytest, predicted_values)

# Plot confusion matrix for Naive Bayes
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="OrRd", fmt="d", xticklabels=np.unique(Ytest), yticklabels=np.unique(Ytest))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix - Naive Bayes')
plt.show()


In [ ]:
score = cross_val_score(NaiveBayes,features,target,cv=5)
score

## **Support Vector Machine (SVM)**

In [ ]:
from sklearn.svm import SVC

SVM = SVC()

SVM.fit(Xtrain,Ytrain)

predicted_values = SVM.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('SVM')
print("SVM's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Ytest, predicted_values)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="OrRd", fmt="d", xticklabels=np.unique(Ytest), yticklabels=np.unique(Ytest))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Cross validation score (SVM)
score = cross_val_score(SVM,features,target,cv=5)
score

## **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression(random_state=2)

LogReg.fit(Xtrain,Ytrain)

predicted_values = LogReg.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('Logistic Regression')
print("Logistic Regression's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Ytest, predicted_values)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="OrRd", fmt="d", xticklabels=np.unique(Ytest), yticklabels=np.unique(Ytest))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Cross validation score (Logistic Regression)
score = cross_val_score(LogReg,features,target,cv=5)
score

## **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=20, random_state=0)
RF.fit(Xtrain,Ytrain)

predicted_values = RF.predict(Xtest)

x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('RF')
print("RF's Accuracy is: ", x)

print(classification_report(Ytest,predicted_values))

In [ ]:

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Ytest, predicted_values)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="OrRd", fmt="d", xticklabels=np.unique(Ytest), yticklabels=np.unique(Ytest))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Cross validation score (Random Forest)
score = cross_val_score(RF,features,target,cv=5)
score

## **Accuracy of each model**

In [ ]:
accuracy_models = dict(zip(model, acc))
for k, v in accuracy_models.items():
    print (k, '-->', v)

In [ ]:
import matplotlib.pyplot as plt

# Assuming accuracy_models is a dictionary containing model names as keys and corresponding accuracies as values

# Plotting the bar graph
plt.figure(figsize=(10, 6))
bars = plt.bar(accuracy_models.keys(), accuracy_models.values(), color='skyblue')

# Adding percentage annotations on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 0.01, f'{yval*100:.2f}%', ha='center', va='bottom', fontsize=10)

plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy of Different Models')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Displaying the plot
plt.show()


# **Hyperparameter Tuning for Random Forest Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


# Define the parameter grid for Randomized Search
param_grid = {
    'n_estimators': np.arange(50, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(2, 25),
    'min_samples_split': np.arange(2, 25),
    'min_samples_leaf': np.arange(2, 25)
}

# Initialize Random Forest Classifier
rf = RandomForestClassifier()

# Initialize RandomizedSearchCV with the Random Forest classifier and parameter grid
rscv_model = RandomizedSearchCV(rf, param_distributions=param_grid, n_iter=100, cv=5, random_state=42)

# Fit RandomizedSearchCV to the training data
rscv_model.fit(Xtrain, Ytrain)

# Get the best estimator (best model) from RandomizedSearchCV
best_estimator = rscv_model.best_estimator_

# Print the best parameters found by RandomizedSearchCV
print("Best Parameters:", rscv_model.best_params_)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

best_rf_model = RandomForestClassifier(n_estimators=192, min_samples_split=17, min_samples_leaf=2, max_depth=18, criterion='entropy', random_state=42)


In [ ]:
best_rf_model.fit(Xtrain, Ytrain)


# **Training with the best parameters**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

# Define the best hyperparameters
best_params = {
    'n_estimators': 192,
    'min_samples_split': 17,
    'min_samples_leaf': 2,
    'max_depth': 18,
    'criterion': 'entropy'
}

# Initialize the Random Forest classifier with the best hyperparameters
best_RF = RandomForestClassifier(**best_params, random_state=0)

# Train the Random Forest classifier using the best hyperparameters
best_RF.fit(Xtrain, Ytrain)

# Make predictions on the test set
predicted_values = best_RF.predict(Xtest)

# Calculate accuracy and append to the accuracy list
x = metrics.accuracy_score(Ytest, predicted_values)
acc.append(x)
model.append('RF')
print("RF's Accuracy is: ", x)






# **Succesfull Prediction using sample data**

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Sample input data
N = 200
P = 50.6
K = 0.29
temperature = 10.603016
humidity = 20.3
ph = 5
rainfall = 110.91

# Create a dictionary with feature names and values
input_data = {
    'N': N,
    'P': P,
    'K': K,
    'temperature': temperature,
    'humidity': humidity,
    'ph': ph,
    'rainfall': rainfall
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame([input_data])

# Extract feature names
feature_names = df.columns.tolist()

# Convert DataFrame to numpy array for prediction
data = df.values

# Make prediction using the trained RandomForestClassifier
prediction = RF.predict(data)
print(prediction)

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Sample input data
N = 50
P = 100
K = 150
temperature = 25.603016
humidity = 60.3
ph = 5
rainfall = 110.91

# Create a dictionary with feature names and values
input_data = {
    'N': N,
    'P': P,
    'K': K,
    'temperature': temperature,
    'humidity': humidity,
    'ph': ph,
    'rainfall': rainfall
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame([input_data])

# Extract feature names
feature_names = df.columns.tolist()

# Convert DataFrame to numpy array for prediction
data = df.values

# Make prediction using the trained RandomForestClassifier
prediction = RF.predict(data)
print(prediction)


In [ ]:
import pickle

pickle_file_path = '/content/drive/My Drive/crop_prediction_model.pkl'

# Open the file in binary write mode ('wb') and dump the RF model into it
with open(pickle_file_path, 'wb') as f:
    pickle.dump(RF, f)


In [ ]:
!pip install flask-ngrok
!pip install pyngrok

# **Change Directory**

In [ ]:
cd/content/drive/MyDrive/Colab Notebooks/data/Flask-deploy-model

# **Import Required Packages**

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template_string
import pickle

# **Create Flask App**

In [ ]:
!ngrok authtoken 2fbc4yti3QB2SA8H3cHWfh1LwXm_6cvcq1QpaKfN4VfGVNtUQ

In [ ]:
from flask import Flask, render_template, request
import pickle
import numpy as np
import requests
from pyngrok import ngrok  # Import ngrok module
from flask import jsonify

app = Flask(__name__)

# Load the trained model from the pickle file
with open('crop_prediction_model.pkl', 'rb') as f:
    model = pickle.load(f)


def predict_crop(N, P, K, temperature, humidity, ph, rainfall):
    # Make prediction
    prediction = model.predict([[N, P, K, temperature, humidity, ph, rainfall]])
    return prediction[0]

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/favicon.ico')
def favicon():
    return '', 204

@app.route('/getprediction', methods=['POST'])
def getprediction():
    # Get input values from the form
    try:
        # Get input values from the form
        N = float(request.form.get('N', 0.0))  # Provide a default value of 0.0 if 'N' is missing or invalid
        P = float(request.form.get('P', 0.0))
        K = float(request.form.get('K', 0.0))
        temperature = float(request.form.get('temperature', 0.0))
        humidity = float(request.form.get('humidity', 0.0))
        ph = float(request.form.get('ph', 0.0))
        rainfall = float(request.form.get('rainfall', 0.0))

        # Make prediction
        prediction = predict_crop(N, P, K, temperature, humidity, ph, rainfall)

        # Return prediction result as JSON
        return jsonify({'output': 'Predicted Crop: {}'.format(prediction)})

        if image_data:
            return jsonify({'output': 'Predicted Crop: {}'.format(prediction), 'image': image_data})
        else:
            return jsonify({'output': 'Predicted Crop: {}'.format(prediction), 'error': 'Image not found for predicted crop'})

    except ValueError:
        return jsonify({'error': 'Invalid input data format'})
    except Exception as e:
        return jsonify({'error': 'Failed to make prediction'})

    # Return the prediction
    return render_template('index.html', output='Predicted Crop: {}'.format(prediction))

if __name__ == "__main__":
    # Connect ngrok tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)


# Keep the Flask app running
app.run()